# Chain

### Flow Engineering: Multi-Step LLM Chains

This architecture builds upon the basic single-call pattern by using **multiple LLM calls in a predefined sequence**.  
Each run of the application follows the same series of LLM interactions—though the inputs and outputs vary with each invocation.

This pattern is often referred to as **flow engineering**.

#### Example: Text-to-SQL Application

Consider a text-to-SQL system, where the user inputs a natural language description of a desired database operation.  
While a single LLM call might be sufficient for basic scenarios, a more robust solution involves a multi-step flow:

---

**Step 1:**  
An **LLM call** that converts the user’s natural language request into a SQL query.  
Inputs:
- User’s natural language question  
- Developer-provided description of the database schema

---

**Step 2:**  
A second **LLM call** that produces a human-readable explanation of the SQL query.  
This helps the user verify that the query matches their intent.

---

**Optional Extensions:**

- **Step 3:** Execute the generated SQL query against the database to retrieve a result table.

- **Step 4:** A third **LLM call** that summarizes the results into a natural language response, directly answering the user’s original question.

---

This sequential approach enables richer, more accurate, and user-friendly interactions—especially for complex use cases.

Let's see an example:

In [6]:
from typing import Annotated, TypedDict

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages

# useful to generate SQL query
model_low_temp = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21", temperature=0.1, model="gpt-4o")
# useful to generate natural language outputs
model_high_temp = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21", temperature=0.7, model="gpt-4o")

class State(TypedDict):
    # to track conversation history
    messages: Annotated[list, add_messages]
    # input
    user_query: str
    # output
    sql_query: str
    sql_explanation: str

class Input(TypedDict):
    user_query: str

class Output(TypedDict):
    sql_query: str
    sql_explanation: str

generate_prompt = SystemMessage(
    """You are a helpful data analyst who answers with SQL queries for users based 
    on their questions."""
)

def generate_sql(state: State) -> State:
    user_message = HumanMessage(state["user_query"])
    messages = [generate_prompt, *state["messages"], user_message]
    res = model_low_temp.invoke(messages)
    return {
        "sql_query": res.content,
        # update conversation history
        "messages": [user_message, res],
    }

explain_prompt = SystemMessage(
    "You are a helpful data analyst who explains SQL queries to users."
)

def explain_sql(state: State) -> State:
    messages = [
        explain_prompt,
        # contains user's query and SQL query from prev step
        *state["messages"],
    ]
    res = model_high_temp.invoke(messages)
    return {
        "sql_explanation": res.content,
        # update conversation history
        "messages": res,
    }

builder = StateGraph(State, input=Input, output=Output)
builder.add_node("generate_sql", generate_sql)
builder.add_node("explain_sql", explain_sql)
builder.add_edge(START, "generate_sql")
builder.add_edge("generate_sql", "explain_sql")
builder.add_edge("explain_sql", END)

graph = builder.compile()

Let's show the graph

In [4]:
graph.get_graph().draw_mermaid_png(output_file_path="../img/sql_graph.png")

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x8b\x00\x00\x01M\x08\x02\x00\x00\x00\xbf\xbd\x0b\xb5\x00\x00\x10\x00IDATx\x9c\xec\x9d\x07X\x14\xc7\xdb\xc0\xe7\xb8;\xee8\xaeq\x1c\xbd#""\nV\x82\x1aL\xc0\x1ec\x8b5\x8aFc\x8c\xb1\xb7\x18{O4\xb6\x98h41\xb1a\x89&\x01\xb1\xc4\x165F\xa3b\xd4\x88\x8a\x8a\x05\x14\x90\xde\xafs\x8d\xef\xc5\xf3\xe3O\x14T\xe2\xee9\xae\xf3{\xee\xe1\xd9\xdbr\xec\xedo\xe7\x9dwg\xe7f9\x15\x15\x15\x88\x801\x1cD\xc0\x1bb\x08w\x88!\xdc!\x86p\x87\x18\xc2\x1db\x08w\xacmH\xab2\x16\xe7\x194\n\xa3Fi2\x19+\x8c\x86W \xd7\xe7\xd9\xd9pmm\x04b\xb6@\xc4v\xf6\xe2#\xebb%C\xcab\xc3\x9d$\xd5\xbdd\xb5Nc\xb2\xb3g\x0b\xc4\x1c\xf8\xb6B\x07\x0ez\x15.\xc6L\xa6\x8a\xc2\xfbZ\x8d\xc2\xc4\x13\xd8d\xa4h\xfcB\xec\xfd\x1b\x0b\xfd\x1a\xd9#\xab\xc0\xa2\xfb\x8a\xd5\xa07\x9f\xdd_\xa4(2\xc8\\m\xe1\xbb\xb9\xfb\xdb\xa1W\x19\xad\xca\x04\xe7Yv\x9a6\xf7\xbe\xae\xf5\xbb\x8e\xa0\n\xd1\x0c\xbd\x86\xae\xfeUzv_\x11|\x93&oJ\x11\xb3(\xc9\xd7\xc3\x99\xc7b\xa1\x8e\x83]8\xb66\x886h4tlg\x9e\xd4\x89\xdb\xa2\x83\x0c1\x97\

Try it

In [7]:
graph.invoke({
  "user_query": "What is the total sales for each product?"
})

{'sql_query': 'Here is the SQL query to calculate the total sales for each product:\n\n```sql\nSELECT \n    product_id,\n    SUM(sales_amount) AS total_sales\nFROM \n    sales_table\nGROUP BY \n    product_id;\n```\n\nMake sure to replace `sales_table` with the name of your table, `product_id` with the column representing the product identifier, and `sales_amount` with the column representing the sales amount in your database.',
 'sql_explanation': 'Let me explain the query step by step:\n\n1. **`SELECT product_id, SUM(sales_amount) AS total_sales`**:\n   - This selects the `product_id` to group the results by each product.\n   - It calculates the total sales for each product using the `SUM()` function, which adds up all the values in the `sales_amount` column for each product.\n\n2. **`FROM sales_table`**:\n   - This specifies the table (`sales_table`) where your sales data is stored. Replace this with the actual table name in your database.\n\n3. **`GROUP BY product_id`**:\n   - This

### Execution Flow of a Multi-Step StateGraph

The process begins with the execution of the `generate_sql` node.  
This step performs two actions:

- Populates the `sql_query` key in the state, which will be part of the final output.
- Updates the `messages` key with new messages generated during this step.

Next, the `explain_sql` node is executed. It uses the previously generated SQL query and:

- Populates the `sql_explanation` key in the state.

Once these nodes have run, the **graph completes execution**, and the final output is returned to the caller.

---

### Input and Output Schema Customization

When defining a `StateGraph`, you can specify **distinct input and output schemas**. This allows for:

- **Input schema**: Defining which parts of the state the user must supply.
- **Output schema**: Specifying what the system returns as final output.

Other keys in the state are used **internally by graph nodes** to hold intermediate data.  
These are not part of the final output but are **accessible in real-time** via the `stream()` method.


Let's see the [Router architecture](LLM_router.ipynb).